In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [20]:
import tensorflow as tf
import numpy as np
import time

sess = tf.Session()

In [3]:
X = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [4]:
# need to initialize weights for many layers in convolution nn
def init_weight(shape):
    init = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init)

def init_bias(shape):
    init = tf.constant(0.1, shape=shape)
    return tf.Variable(init)

In [ ]:
a = init_bias([1, 5])
b = init_weight([2, 2])


sess.run(tf.global_variables_initializer())
sess.run([a, b])

In [5]:
# X is 4d tensor with dimensions: [batch, image_height, image_width, image_channels]
# W is 4d tensor with dimensions: [window_height, window_width, image_channels, nextlayer_channels]
# strides is teh stride of the 4 dimensions of X
def conv2d(X, W):
    return tf.nn.conv2d(X, W, strides=[1, 1, 1, 1], padding='SAME') 
# first and last stride # are always 1, first to make sure no data points in batch are skipped, idk what second does

def max_pool_2x2(Z):
    return tf.nn.max_pool(Z, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 
# ksize is window size of pooling, strides is set so we dont overlap windows

def relu(Z):
    return tf.nn.relu(Z)

def dropout(Z, prob):
    return tf.nn.dropout(Z, prob)

In [9]:
X_image = tf.reshape(X, [-1, 28, 28, 1])

In [10]:
W_1 = init_weight([5, 5, 1, 32])
b_1 = init_bias([32])

In [11]:
l_1 = conv2d(X_image, W_1) + b_1
r_1 = relu(l_1)
p_1 = max_pool_2x2(r_1) # this reduces each dimension by 2, so image is 14x14

In [12]:
W_2 = init_weight([5, 5, 32, 64])
b_2 = init_bias([64])

In [13]:
l_2 = conv2d(p_1, W_2) + b_2
r_2 = relu(l_2)
p_2 = max_pool_2x2(r_2) # this reduces each dimension by 2, so image is 7x7

In [14]:
volume = 7*7*64
W_3 = init_weight([volume, 1024])
b_3 = init_bias([1024])

In [15]:
p_2_flat = tf.reshape(p_2, [-1, volume]) # I wonder how it gets the dimensions right
z_4 = tf.matmul(p_2_flat, W_3) + b_3
r_4 = relu(z_4)
# [batch, ih, iw, ic]
# [] data point
# () vertical dimension
# {} horizontal dimension
# [] depth dimension
# [ [({col1row1[values along depth]col1row2[values along depth]}{[][]})({[][]}{[][]})], [({}{})({}{})]]

In [16]:
keep_prob = tf.placeholder(tf.float32)
dropped = dropout(r_4, keep_prob)

In [17]:
W_4 = init_weight([1024, 10])
b_4 = init_bias([10])

pred = tf.matmul(dropped, W_4) + b_4

In [18]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=pred))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    steps = 20000
    start = time.time()
    for i in range(steps):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            time_per_100 = time.time() - start
            start = time.time()
            train_accuracy = accuracy.eval(feed_dict={X: batch[0], y_true: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g, took %.2f secs' % (i, train_accuracy, time_per_100))
        train_step.run(feed_dict={X: batch[0], y_true: batch[1], keep_prob: 0.5})
        
    print('test accuracy %g' % accuracy.eval(feed_dict={
      X: mnist.test.images, y_true: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.1, took 0.00 secs
step 100, training accuracy 0.92, took 9.93 secs
step 200, training accuracy 0.84, took 9.84 secs
step 300, training accuracy 0.88, took 9.50 secs
step 400, training accuracy 0.94, took 9.45 secs
step 500, training accuracy 0.9, took 9.41 secs
step 600, training accuracy 0.98, took 9.42 secs
step 700, training accuracy 0.96, took 9.99 secs
step 800, training accuracy 0.92, took 9.49 secs
step 900, training accuracy 0.98, took 9.39 secs
step 1000, training accuracy 1, took 9.75 secs
step 1100, training accuracy 1, took 9.44 secs
step 1200, training accuracy 0.96, took 9.46 secs
step 1300, training accuracy 0.96, took 9.50 secs
step 1400, training accuracy 0.96, took 9.70 secs
step 1500, training accuracy 0.98, took 9.46 secs
step 1600, training accuracy 0.94, took 9.58 secs
step 1700, training accuracy 0.94, took 9.59 secs
step 1800, training accuracy 0.96, took 9.89 secs
step 1900, training accuracy 0.94, took 9.40 secs
step 2000, training 